In [ ]:
import tensorflow as tf#1.x vesion
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,Conv2D, MaxPooling3D, Conv3D, MaxPooling2D, TimeDistributed,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, Xception, InceptionV3,InceptionResNetV2



import tensorflow_addons as tfa




import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


import matplotlib.pyplot as plt
import numpy as np

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32



In [ ]:
img_size = (150,150)
X = np.load("../../Disease Image/COVID CXR/covid_org6000_150_X.npy")
Y = np.load("../../Disease Image/COVID CXR/covid_org6000_150_Y.npy")

In [ ]:
from random import sample

In [ ]:

index = [i for i in range(X.shape[0])]
np.random.shuffle(index) 


X2 = X[index]/255.0
Y2 = Y[index]

In [ ]:
Y3 = tf.keras.utils.to_categorical(Y2, 2)

In [ ]:
def activation_block(x):
    x = layers.Activation("gelu")(x)
    return layers.BatchNormalization()(x)


def conv_stem(x, filters: int, patch_size: int):
    x = layers.Conv2D(filters, kernel_size=patch_size, strides=patch_size)(x)
    return activation_block(x)


def conv_mixer_block(x, filters: int, kernel_size: int):
    # Depthwise convolution.
    x0 = x
    x = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(x)
    x = layers.Add()([activation_block(x), x0])  # Residual.

    # Pointwise convolution.
    x = layers.Conv2D(filters, kernel_size=1)(x)
    x = activation_block(x)

    return x


def get_conv_mixer_256_8(
    image_size=150, filters=256, depth=8, kernel_size=5, patch_size=16, num_classes=2
):

    inputs = keras.Input((image_size, image_size, 3))


    # Extract patch embeddings.
    x = conv_stem(inputs, filters, patch_size)

    # ConvMixer blocks.
    for _ in range(depth):
        x = conv_mixer_block(x, filters, kernel_size)

    # Classification block.
    x = layers.GlobalAvgPool2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs, outputs)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=10)
earlystop = EarlyStopping(monitor='val_accuracy', patience=15)

In [ ]:
seed = 101
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvacc = []
cvauc = []

i = 1
for train, test in kfold.split(X2,Y2):
    model = get_conv_mixer_256_8()
    optimizer = tfa.optimizers.AdamW(
    learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    filepath = "../Data/models/150_ConvMix_{0}_lung_16patch.h5".format(str(i))
    if os.path.exists(filepath):
        os.remove(filepath)
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1,monitor='val_accuracy',mode='max'
    )
    

    model.fit(X2[train], Y3[train], validation_split=0.2,epochs=300, batch_size=32, verbose=1,shuffle=True,callbacks=[checkpoint,earlystop])
    model.load_weights(filepath)
    # evaluate the model
    evals = model.evaluate(X2[test], Y3[test], verbose=1)
    y_pred = model.predict(X2[test])
    auc = roc_auc_score(Y3[test],y_pred)
    with open('../Data/models/50_ConvMixer_{0}_lung_16patch.csv'.format(str(i)),'w') as file:
        file.write('true_label,predict_label'+'\n')
        for a,b in zip(Y2[test],y_pred):
            file.write(str(a) + ','+str(np.argmax(b))+'\n')
    acc = evals[1]
    
    
    print("%s: %.2f%%" % (model.metrics_names[1], acc))
    print("auc: %.2f"%(auc))

    cvacc.append(acc*100)
    cvauc.append(auc)

    i+=1
print("mean acc %.2f%% (+/- %.2f%%)" % (np.mean(cvacc), np.std(cvacc)))
print("mean auc %.2f%% (+/- %.2f%%)" % (np.mean(cvauc), np.std(cvauc)))


In [ ]:
cvacc

In [ ]:
np.mean(cvacc)

In [ ]:
cvauc

In [ ]:
np.mean(cvauc)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,roc_auc_score,recall_score,precision_score

In [ ]:
def calculate_metric(gt, pred): 
    pred2 = []
    for i in pred:
        if i>0.5:
            pred2.append(1)
        else:
            pred2.append(0)
    confusion = confusion_matrix(gt,pred2)
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    acc = (TP+TN)/float(TP+TN+FP+FN)
    auc = roc_auc_score(gt,pred)
    precision = TP/float(TP+FP)
    f1_score_ = f1_score(gt,pred2)
    sensitivity = TP / float(TP+FN)
    specificity = TN / float(TN+FP)
    
    return round(acc,4),round(auc,4),round(f1_score_,4), round(precision,4),round(sensitivity,4), round(specificity,4)

In [ ]:
df1 = pd.read_csv('../Data/models/50_ConvMixer_1_lung_16patch.csv')


calculate_metric(df1['true_label'],df1['predict_label'])

In [ ]:
df2 = pd.read_csv('../Data/models/50_ConvMixer_2_lung_16patch.csv')


calculate_metric(df2['true_label'],df2['predict_label'])

In [ ]:
df3 = pd.read_csv('../Data/models/50_ConvMixer_3_lung_16patch.csv')

calculate_metric(df3['true_label'],df3['predict_label'])

In [ ]:
df4 = pd.read_csv('../Data/models/50_ConvMixer_4_lung_16patch.csv')


calculate_metric(df4['true_label'],df4['predict_label'])

In [ ]:
df5 = pd.read_csv('../Data/models/50_ConvMixer_5_lung_16patch.csv')


calculate_metric(df5['true_label'],df5['predict_label'])